## Undercatch Correction ##

Implement undercatch corrections to station data


### Define input file locations ###

In [10]:
import xarray as xr
import pandas as pd
import os
from geopy.distance import geodesic
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import glob
import sys
sys.path.append('../scripts')
from undercatch_processing import *

# add autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define input file locations ###

In [2]:
# Input Paths
input_path = '/Users/dcasson/Data/pems/'
input_station_data_path = Path(input_path,'station_data')

#Input station data
cdn_complete_stations = Path(input_station_data_path,'Station Inventory EN.csv')
gsod_stations = Path(input_station_data_path,'isd-history.csv')
ghcnd_stations = Path(input_station_data_path,'ghcnd-stations.csv')

# Input SC-Earth data
undercatch_stations_nc = Path(input_path,'sc_earth','undercatch_stations.nc')
reanalysis_data_path = Path(input_path,'era5')

# Smith, 2019 data
smith_2019_stations = Path(input_station_data_path,'EN_StationCatalogue_2019.csv')
smith_2019_hourly_path = Path(input_path,'smith_2019/hourly_data/')
smith_2019_daily_path = Path(input_path,'smith_2019/daily_data/')

output_path = '/Users/dcasson/Data/pems/undercatch/'
sc_earth_path = Path(output_path,'sc_earth')
interim_station_path = Path(output_path,'interim_stations')
reanalysis_station_path = Path(output_path,'reanalysis_stations')
merged_station_path = Path(output_path,'merged_stations')
undercatch_station_path = Path(output_path,'undercatch_stations_test')
plot_path = Path(output_path,'plots')


first_filter_year = 2004
reanalysis_variable = 'windspd'

#Output files
cdn_year_filtered_stations = Path(output_path,'cdn_station_inventory.csv')
undercatch_stations_csv = Path(output_path,'undercatch_from_nc.csv')
undercatch_stations_update_csv = Path(output_path,'undercatch_stations_update.csv')
evaluation_stations_csv = Path(output_path,'evaluation_stations.csv')

# Make output directories
os.makedirs(smith_2019_daily_path, exist_ok=True)
os.makedirs(interim_station_path, exist_ok=True)
os.makedirs(reanalysis_station_path, exist_ok=True)
os.makedirs(merged_station_path, exist_ok=True)
os.makedirs(sc_earth_path, exist_ok=True)
os.makedirs(undercatch_station_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)


### Filter Canadian Large Dataset, to remove old stations

In [3]:
def filter_stations_by_year(input_csv_path, output_csv_path, year):
    # Load the CSV file with appropriate delimiter and skip initial lines
    stations_df = pd.read_csv(input_csv_path, delimiter=',', skiprows=3)
    
    # Filter out rows where 'DLY Last Year' is before the specified year
    filtered_df = stations_df[stations_df['DLY Last Year'] >= year]
    
    # Save the filtered DataFrame to a new CSV file
    filtered_df.to_csv(output_csv_path, index=False)
    print(f"Filtered stations saved to {output_csv_path}")

filter_stations_by_year(cdn_complete_stations, cdn_year_filtered_stations, first_filter_year)

Filtered stations saved to /Users/dcasson/Data/pems/undercatch/cdn_station_inventory.csv


### Read stations from SC-Earth

In [4]:
def read_undercatch_stations(nc_file_path, output_csv_path):
    # Open the netCDF file using xarray
    ds = xr.open_dataset(nc_file_path)
    
    # Extract variables
    station_ids = ds['station_ID'].values
    latitudes = ds['latitude'].values
    longitudes = ds['longitude'].values
    elevations = ds['elevation'].values
    
    # Process station_IDs to remove prefix
    processed_station_ids = [sid.split('_')[1][:5] for sid in station_ids]
    
    # Create DataFrame
    df = pd.DataFrame({
        'full_station_ID':station_ids,
        'station_ID': processed_station_ids,
        'latitude': latitudes,
        'longitude': longitudes,
        'elevation': elevations
    })
    
    # Output to CSV
    df.to_csv(output_csv_path, index=False)
    print(f"Undercatch stations saved to {output_csv_path}")

read_undercatch_stations(undercatch_stations_nc, undercatch_stations_csv)

Undercatch stations saved to /Users/dcasson/Data/pems/undercatch/undercatch_from_nc.csv


### Merge station metadata

In [5]:
def join_datasets(file1_path, file2_path, join_column_file1, join_column_file2, output_path, join_type='inner'):
    # Load the datasets
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)

    # Ensure the columns for joining are strings and strip trailing .0 if present
    df1[join_column_file1] = df1[join_column_file1].astype(str).str.rstrip('.0')
    df2[join_column_file2] = df2[join_column_file2].astype(str).str.rstrip('.0')

    # Perform the join
    merged_df = pd.merge(df1, df2, left_on=join_column_file1, right_on=join_column_file2, how=join_type)

    # Save the result to CSV
    merged_df.to_csv(output_path, index=False)
    print(f"Joined {file1_path} and {file2_path} saved to {output_path}")

#join_datasets(undercatch_stations_csv, cdn_year_filtered_stations, 'station_ID', 'WMO ID', undercatch_stations_update_csv)

#join_datasets(undercatch_stations_update_csv,smith_2019_stations,'Climate ID', 'StationID', evaluation_stations_csv)

tuolumne_prcp = '/Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset.csv'
ghcnd = '/Users/dcasson/Data/pems/station_data/ghcnd-stations.csv'
gsod = '/Users/dcasson/Data/pems/station_data/isd-history.csv'

join_datasets(tuolumne_prcp,ghcnd,'station_ID','StationID','/Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset_ghcnd.csv')
join_datasets(tuolumne_prcp,gsod,'station_ID','USAF','/Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset_gsod.csv')

Joined /Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset.csv and /Users/dcasson/Data/pems/station_data/ghcnd-stations.csv saved to /Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset_ghcnd.csv
Joined /Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset.csv and /Users/dcasson/Data/pems/station_data/isd-history.csv saved to /Users/dcasson/Data/gpep/tuolumne/data_prep/stations/prcp_subset_gsod.csv


In [58]:
import pandas as pd
from geopy.distance import geodesic

# Load GHCNd stations metadata
ghcnd_stations = pd.read_csv('/Users/dcasson/Data/gpep/tuolumne/data_prep/stations/tuolumne_stations_ghcnd.csv')  # Adjust the path as needed

# Load CDEC stations metadata
cdec_stations = pd.read_csv('/Users/dcasson/Data/pems/station_data/CDEC.csv')  # Adjust the path as needed

# Function to calculate distance between two coordinates
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).km

# Create an empty list to store matches
matches = []

# Iterate through each GHCNd station
for idx, ghcnd_station in ghcnd_stations.iterrows():
    ghcnd_coords = (ghcnd_station['latitude'], ghcnd_station['longitude'])
    closest_station = None
    min_distance = float('inf')

    # Iterate through each CDEC station
    for _, cdec_station in cdec_stations.iterrows():
        cdec_coords = (cdec_station['Latitude'], cdec_station['Longitude'])
        distance = calculate_distance(ghcnd_coords, cdec_coords)

        # Check if the current CDEC station is the closest match
        if distance < min_distance:
            min_distance = distance
            closest_station = cdec_station

    # If the closest station is within a reasonable distance, consider it a match
    if min_distance < 3.0:  # Adjust the distance threshold as needed
        matches.append({
            'ghcnd_id': ghcnd_station['station_ID'],
            'ghcnd_name': ghcnd_station['StationName'],
            'cdec_id': closest_station['ID'],
            'cdec_name': closest_station['Station Name'],
            'cdec_operator': closest_station['Operator'],
            'distance_km': min_distance
        })

# Convert matches to a DataFrame for easy analysis
matches_df = pd.DataFrame(matches)

# Save the matches to a CSV file
matches_df.to_csv('/Users/dcasson/Data/gpep/tuolumne/data_prep/stations/matched_stations.csv', index=False)

print('Matching complete. Results saved to matched_stations.csv')

Matching complete. Results saved to matched_stations.csv


In [7]:
def extract_sc_earth_station_data(nc_file_path, csv_file_path, output_dir):
    # Load the netCDF file
    ds = xr.open_dataset(nc_file_path)
    
    # Load the CSV file
    stations_df = pd.read_csv(csv_file_path)
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Create a dictionary to map station_ID to station_number
    station_id_map = {str(ds['station_ID'][i].values): i for i in range(len(ds['station_ID']))}
    
    # Iterate through the stations in the CSV file
    for _, row in stations_df.iterrows():
        climate_id = row['Climate ID']
        if not pd.isna(climate_id):
            station_id = row['full_station_ID']
            station_number = station_id_map.get(station_id, None)
            
            if station_number is not None:
                # Extract data for the station
                time = ds['time'].values
                prcp = ds['prcp'].isel(station_number=station_number).values
                tmean = ds['tmean'].isel(station_number=station_number).values
                wind = ds['wind'].isel(station_number=station_number).values
                
                # Create a DataFrame for the station data
                station_data = pd.DataFrame({
                    'time': time,
                    'prcp': prcp,
                    'tmean': tmean,
                    'wind': wind
                })
                
                # Define the output file path
                output_file_path = os.path.join(output_dir, f'station_{str(climate_id)}.csv')
                
                # Save the DataFrame to a CSV file
                station_data.to_csv(output_file_path, index=False)
                print(f"Station data saved to {output_file_path}")

extract_sc_earth_station_data(undercatch_stations_nc, undercatch_stations_update_csv, sc_earth_path)


Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3020610.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3012050.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3050519.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3031480.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3050778.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3051R4R.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3026KNQ.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3024925.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3025297.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3011892.csv
Station data saved to /Users/dcasson/Data/pems/undercatch/sc_earth/station_3053536.csv
Station data saved to /Users/dcasson/Data/p

### Extract reanalysis data for undercatch stations

In [8]:
def load_points(csv_path):
    """Load points from CSV file."""
    df = pd.read_csv(csv_path)
    # Convert station_ID to string without tuple notation
    df['station_ID'] = df['station_ID'].apply(lambda x: str(x).strip("(),'"))
    return df

def find_nearest_grid_point(ds, orig_lat, orig_lon):
    """Find the nearest grid point in the dataset for the given latitude and longitude."""
    latitudes = ds['latitude'].values
    longitudes = ds['longitude'].values
    abs_diff_lat = abs(latitudes - orig_lat)
    abs_diff_lon = abs(longitudes - orig_lon)
    combined_diff = abs_diff_lat + abs_diff_lon
    min_diff_idx = combined_diff.argmin()
    nearest_lat_idx, nearest_lon_idx = np.unravel_index(min_diff_idx, latitudes.shape)
    return nearest_lat_idx, nearest_lon_idx

def process_reanalysis_station(ds, station_id, station_name, orig_lat, orig_lon, variable, output_dir):
    """Extract raster values for a specific variable at a specified station and save to CSV."""
    if variable not in ds:
        raise ValueError(f"Variable {variable} not found in the dataset")

    # Find the nearest grid point using the lat-lon variables in the dataset
    nearest_lat_idx, nearest_lon_idx = find_nearest_grid_point(ds, orig_lat, orig_lon)
    
    # Extract the data for the nearest grid point
    point_data = ds[variable].isel(rlat=nearest_lat_idx, rlon=nearest_lon_idx).to_dataframe().reset_index()

    # Add station ID, original latitude and longitude
    point_data['station_ID'] = station_id
    point_data['latitude'] = orig_lat
    point_data['longitude'] = orig_lon

    # Calculate daily averages
    point_data['time'] = pd.to_datetime(point_data['time'])
    numeric_cols = point_data.select_dtypes(include='number').columns
    daily_data = point_data.resample('D', on='time')[numeric_cols].mean().reset_index()
    daily_data['station_ID'] = station_id
    daily_data['latitude'] = orig_lat
    daily_data['longitude'] = orig_lon

    # Save hourly results to CSV
    hourly_output_path = os.path.join(output_dir, f"station_{str(station_name)}_hourly.csv")
    if os.path.exists(hourly_output_path):
        existing_df = pd.read_csv(hourly_output_path, parse_dates=['time'])
        combined_df = pd.concat([existing_df, point_data], ignore_index=True)
        combined_df.to_csv(hourly_output_path, index=False)
    else:
        point_data.to_csv(hourly_output_path, index=False)

    # Save daily average results to CSV
    daily_output_path = os.path.join(output_dir, f"station_{str(station_name)}_daily.csv")
    if os.path.exists(daily_output_path):
        existing_df = pd.read_csv(daily_output_path, parse_dates=['time'])
        combined_df = pd.concat([existing_df, daily_data], ignore_index=True)
        combined_df.to_csv(daily_output_path, index=False)
    else:
        daily_data.to_csv(daily_output_path, index=False)

def extract_reanalysis_station_data(csv_path, nc_dir, output_dir, variable):
    """Extract data from all NetCDF files in a directory for each station and save results."""
    points_df = load_points(csv_path)

    # Load and concatenate all NetCDF files
    nc_files = [os.path.join(nc_dir, f) for f in os.listdir(nc_dir) if f.endswith('.nc')]
    ds = xr.open_mfdataset(nc_files, combine='by_coords')

    for index, row in points_df.iterrows():
        station_id = row['station_ID']
        station_name = str(row['Climate ID']),
        station_name = station_name[0].replace(' ','')
        orig_lat = row['latitude']
        orig_lon = row['longitude']
        print(f"Processing station {station_name}")
        process_reanalysis_station(ds, station_id, station_name, orig_lat, orig_lon, variable, output_dir)

# Execute extraction 
extract_reanalysis_station_data(undercatch_stations_update_csv, reanalysis_data_path, reanalysis_station_path, reanalysis_variable)

Processing station 3020610


ValueError: operands could not be broadcast together with shapes (25,) (41,) 

### Merge reanalysis data with SC-Earth data

In [12]:

def load_stations(csv_file_path):
    """Load station data from a CSV file."""
    stations_df = pd.read_csv(csv_file_path)
    stations = stations_df[['Climate ID', 'latitude', 'longitude']].values
    return stations

def extract_reanalysis_at_stations_and_compute_means(ds, stations, output_dir):
    """Extract data from the dataset for all stations and compute means."""
    for station in stations:
        climate_id, lat, lon = station
        point = ds.sel(latitude=lat, longitude=lon, method='nearest').to_dataframe().reset_index()
        point['time'] = pd.to_datetime(point['time'])
        
        hourly_mean = point
        daily_mean = point.resample('D', on='time').mean().reset_index()
        
        hourly_mean.to_csv(f'{output_dir}/station_{climate_id}_hourly.csv', index=False)
        daily_mean.to_csv(f'{output_dir}/station_{climate_id}_daily.csv', index=False)
        print(f"Processed station {climate_id}")
 
stations = load_stations(undercatch_stations_update_csv)
nc_file_paths = glob.glob(f'{reanalysis_data_path}/*.nc')
ds = xr.open_mfdataset(nc_file_paths, combine='by_coords')

extract_reanalysis_at_stations_and_compute_means(ds, stations, reanalysis_station_path)

Processed station 3020610
Processed station 3012050
Processed station 3050519
Processed station 3031480
Processed station 3050778
Processed station 3051R4R
Processed station 3026KNQ
Processed station 3024925
Processed station 3025297
Processed station 3011892
Processed station 3053536
Processed station 3034795
Processed station 3030720
Processed station 3036205
Processed station 3023200
Processed station 3023740
Processed station 3035208
Processed station 3031092
Processed station 3031093
Processed station 1176755
Processed station 3017282
Processed station 3015523


In [13]:
def merge_daily_wind_reanalysis_to_sc_earth_files(daily_dir, station_dir, output_dir):
    """
    Merges the "windspd" variable from daily CSV files into station CSV files based on matching station IDs.
    
    Parameters:
    - daily_dir: str, path to the directory containing daily CSV files.
    - station_dir: str, path to the directory containing station CSV files.
    - output_dir: str, path to the directory to save the merged CSV files.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Get list of daily and station files
    daily_files = {os.path.splitext(f)[0]: f for f in os.listdir(daily_dir) if f.endswith('daily.csv')}
    # Remove the daily from all daily files
    daily_files = {k.replace('_daily',''): v for k,v in daily_files.items()}

    station_files = {os.path.splitext(f)[0]: f for f in os.listdir(station_dir) if f.endswith('.csv')}
    
    # Find matching station IDs
    matching_ids = set(daily_files.keys()) & set(station_files.keys())
    
    for station_id in matching_ids:
        daily_file_path = os.path.join(daily_dir, daily_files[station_id])
        station_file_path = os.path.join(station_dir, station_files[station_id])
        output_file_path = os.path.join(output_dir, f'{station_id}_sc_earth_with_wind.csv')
        
        # Load the CSV files
        daily_df = pd.read_csv(daily_file_path)
        station_df = pd.read_csv(station_file_path)

        # Ensure the 'time' columns are in the same format
        daily_df['time'] = pd.to_datetime(daily_df['time'])
        station_df['time'] = pd.to_datetime(station_df['time'])

        # Merge the dataframes on the 'time' column
        merged_df = pd.merge(station_df, daily_df[['time', 'windspd']], on='time', how='left')

        # Rename the column to 'reanalysis_wind'
        merged_df.rename(columns={'windspd': 'reanalysis_wind'}, inplace=True)

        # Save the merged DataFrame to a new CSV file
        merged_df.to_csv(output_file_path, index=False)
        print(f"Saved merged data to {output_file_path}")

merge_daily_wind_reanalysis_to_sc_earth_files(reanalysis_station_path, sc_earth_path,interim_station_path)

Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3020610_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3012050_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3023200_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3025297_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3030720_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3011892_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_1176755_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3051R4R_sc_earth_with_wind.csv
Saved merged data to /Users/dcasson/Data/pems/undercatch/interim_stations/station_3031093_sc_ear

In [15]:
def convert_txt_to_csv(txt_file, csv_file):
    # Read the .txt file
    with open(txt_file, 'r') as file:
        lines = file.readlines()

    # Assume the first non-empty line is the header and skip the second header line
    header = next(line for line in lines if line.strip())
    lines = lines[lines.index(header) + 2:]  # Skip the header line and the next line

    # Write the content to a .csv file with the extracted header
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header.split())

        for line in lines:
            if line.strip():
                writer.writerow(line.split())

def calculate_daily_averages_from_eccc_paper(hourly_data_file, daily_data_file):

    # Define columns
    columns = ["YYYYMMDDThhmm", "Unadj_P(mm)", "Tair(C)", "Wind(m/s)", "Wind_Flag", 
               "CE", "UTF_Adj_P(mm)", "CODECON(mm)", "UTF_Adj+CODECON_P(mm)", "Adj_Flag"]
    
    # Read the data file
    df = pd.read_csv(hourly_data_file, delim_whitespace=True, skiprows=2, names=columns)
    
    # Replace -99999 with NaN
    df.replace(-99999, np.nan, inplace=True)

    # Read date column to datetime
    df['YYYYMMDDThhmm'] = pd.to_datetime(df['YYYYMMDDThhmm'], format='%Y%m%dT%H%M')

    # Shift the date column back by 1 hour
    df['YYYYMMDDThhmm'] = df['YYYYMMDDThhmm'] + pd.Timedelta(hours=1)

    # Write YYYYMMDD to Date column
    df['Date'] = df['YYYYMMDDThhmm'].dt.strftime('%Y%m%d')
    
    # Extract date part from datetime
    #df['Date'] = df['YYYYMMDDThhmm'].str[:8]
    
    # Calculate daily averages, ignoring flags
    daily_avg = df.groupby('Date').agg({
        "Unadj_P(mm)": "sum",
        "Tair(C)": "mean",
        "Wind(m/s)": "mean",
        "CE": "mean",
        "UTF_Adj_P(mm)": "sum",
        "CODECON(mm)": "mean",
        "UTF_Adj+CODECON_P(mm)": "sum"
    }).reset_index()
    
    # Write daily averages to a CSV file
    daily_avg.to_csv(daily_data_file, index=False)
    print(f"Daily averages have been written to {daily_data_file}")


def read_station_id_list(csv_file, column_name):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Return a list of the entries in the specified column
    return df[column_name].tolist()

def select_and_convert_raw_data_stations(station_id_list,hourly_path, daily_path):
    for station in station_id_list:
        station = str(station)
        for file_name in os.listdir(hourly_path):
            if station in file_name and file_name.endswith('.txt'):
                # Construct the full file paths
                hourly_file = os.path.join(hourly_path, file_name)
                daily_file = os.path.join(daily_path, file_name)

                #Update .txt to .csv
                daily_file = daily_file.replace('.txt', '.csv')

                # Define the destination .csv file name
                calculate_daily_averages_from_eccc_paper(hourly_file, daily_file)
                print(f"Converted and copied: {hourly_file} to {daily_file}")


def merge_obs_and_model_csv_files(station_id_list, smith_2019_daily_path, gsod_path, merged_station_path,station_name_list):

    for station, name in zip(station_id_list,station_name_list):
        station = str(station)
        for file_name in os.listdir(smith_2019_daily_path):
            if station in file_name and file_name.endswith('.csv'):
                # Construct the full file paths
                obs_file = os.path.join(smith_2019_daily_path, file_name)
                model_file = os.path.join(gsod_path, f'station_{station}_sc_earth_with_wind.csv')
                df1 = pd.read_csv(obs_file)
                df2 = pd.read_csv(model_file)

                # Rename the date columns to a common name
                df1 = df1.rename(columns={'Date': 'date'})
                df2 = df2.rename(columns={'time': 'date'})

                # Strip any leading/trailing whitespace from the date column in df1
                df1['date'] = df1['date'].astype(str).str.strip()
                
                # Convert the 'YYYYMMDD' date format in df1 to a proper datetime format
                df1['date'] = pd.to_datetime(df1['date'])
                
                # Convert the date column in df2 to datetime format
                df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')
                
                # Convert all other columns to numeric, coercing errors to NaN
                for col in df1.columns:
                    if col != 'date':
                        df1[col] = pd.to_numeric(df1[col], errors='coerce')
                
                for col in df2.columns:
                    if col != 'date':
                        df2[col] = pd.to_numeric(df2[col], errors='coerce')
                
                # Merge the dataframes on the 'date' column
                merged_df = pd.merge(df1, df2, on='date', how='outer')
                
                # Drop rows where there are NaN values in any of the columns
                merged_df = merged_df.dropna()
                
                # Save the merged DataFrame to a new CSV file
                merged_df.to_csv(Path(merged_station_path, f'{name}.csv'), index=False)
                print(f"Merged data saved to {merged_station_path}/{name}.csv")

station_id_list = read_station_id_list(evaluation_stations_csv, 'Climate ID')
station_name_list = read_station_id_list(evaluation_stations_csv, 'Name')
station_name_list = [s.replace(' ', '') for s in station_name_list]


select_and_convert_raw_data_stations(station_id_list, smith_2019_hourly_path, smith_2019_daily_path)
merge_obs_and_model_csv_files(station_id_list, smith_2019_daily_path, interim_station_path, merged_station_path, station_name_list)


/var/folders/w4/qfhdls_x5jj48jhgnhtfv1lm0000gn/T/ipykernel_3247/1725123043.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(hourly_data_file, delim_whitespace=True, skiprows=2, names=columns)


Daily averages have been written to /Users/dcasson/Data/pems/smith_2019/daily_data/3050519_UTF_hly_prec.csv
Converted and copied: /Users/dcasson/Data/pems/smith_2019/hourly_data/3050519_UTF_hly_prec.txt to /Users/dcasson/Data/pems/smith_2019/daily_data/3050519_UTF_hly_prec.csv


/var/folders/w4/qfhdls_x5jj48jhgnhtfv1lm0000gn/T/ipykernel_3247/1725123043.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(hourly_data_file, delim_whitespace=True, skiprows=2, names=columns)


Daily averages have been written to /Users/dcasson/Data/pems/smith_2019/daily_data/3050778_UTF_hly_prec.csv
Converted and copied: /Users/dcasson/Data/pems/smith_2019/hourly_data/3050778_UTF_hly_prec.txt to /Users/dcasson/Data/pems/smith_2019/daily_data/3050778_UTF_hly_prec.csv


/var/folders/w4/qfhdls_x5jj48jhgnhtfv1lm0000gn/T/ipykernel_3247/1725123043.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(hourly_data_file, delim_whitespace=True, skiprows=2, names=columns)


...

Daily averages have been written to /Users/dcasson/Data/pems/smith_2019/daily_data/3035208_UTF_hly_prec.csv
Converted and copied: /Users/dcasson/Data/pems/smith_2019/hourly_data/3035208_UTF_hly_prec.txt to /Users/dcasson/Data/pems/smith_2019/daily_data/3035208_UTF_hly_prec.csv


/var/folders/w4/qfhdls_x5jj48jhgnhtfv1lm0000gn/T/ipykernel_3247/1725123043.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(hourly_data_file, delim_whitespace=True, skiprows=2, names=columns)


Daily averages have been written to /Users/dcasson/Data/pems/smith_2019/daily_data/1176755_UTF_hly_prec.csv
Converted and copied: /Users/dcasson/Data/pems/smith_2019/hourly_data/1176755_UTF_hly_prec.txt to /Users/dcasson/Data/pems/smith_2019/daily_data/1176755_UTF_hly_prec.csv


/var/folders/w4/qfhdls_x5jj48jhgnhtfv1lm0000gn/T/ipykernel_3247/1725123043.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(hourly_data_file, delim_whitespace=True, skiprows=2, names=columns)


Daily averages have been written to /Users/dcasson/Data/pems/smith_2019/daily_data/3015523_UTF_hly_prec.csv
Converted and copied: /Users/dcasson/Data/pems/smith_2019/hourly_data/3015523_UTF_hly_prec.txt to /Users/dcasson/Data/pems/smith_2019/daily_data/3015523_UTF_hly_prec.csv
Merged data saved to /Users/dcasson/Data/pems/undercatch/merged_stations/BANFFCS.csv
Merged data saved to /Users/dcasson/Data/pems/undercatch/merged_stations/BOWVALLEY.csv
Merged data saved to /Users/dcasson/Data/pems/undercatch/merged_stations/JASPERWARDEN.csv
Merged data saved to /Users/dcasson/Data/pems/undercatch/merged_stations/PINCHERCREEKCLIMATE.csv
Merged data saved to /Users/dcasson/Data/pems/undercatch/merged_stations/REVELSTOKEAIRPORTAUTO.csv
Merged data saved to /Users/dcasson/Data/pems/undercatch/merged_stations/ROCKYMTNHOUSE(AUT).csv


In [1]:
def calculate_undercatch_for_gsod_stations(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            
            # Ensure required columns are present
            if all(col in df.columns for col in ['wind', 'tmean', 'prcp']):
                # Calculate CE and corrected precipitation
                df['CE'] = df.apply(lambda row: calculate_CE(row['wind'], row['tmean']), axis=1)
                df['corrected_prcp'] = df.apply(lambda row: apply_undercatch(row['prcp'], row['CE']), axis=1)
                #Update so that if corrected_prcp is 0, set CE to 1
                #df.loc[df['corrected_prcp'] == 0, 'CE'] = 1
                
                # Save the updated DataFrame back to the CSV file
                df.to_csv(file_path, index=False)
                print(f"Undercatch calculated for {file_path}")

# Process the CSV files in the directory
#calculate_undercatch_for_gsod_stations(output_station_path)

def calculate_undercatch_for_gsod_stations_with_reanalysis(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            
            # Ensure required columns are present
            if all(col in df.columns for col in ['wind', 'tmean', 'prcp','reanalysis_wind']):
                # Calculate CE and corrected precipitation
                df['CE_sc_earth'] = df.apply(lambda row: calculate_CE(row['wind'], row['tmean']), axis=1)
                df['corrected_prcp_sc_earth'] = df.apply(lambda row: apply_undercatch(row['prcp'], row['CE_sc_earth']), axis=1)

                df['CE_sc_reanalysis'] = df.apply(lambda row: calculate_CE(row['reanalysis_wind'], row['tmean']), axis=1)
                df['corrected_prcp_sc_reanalysis'] = df.apply(lambda row: apply_undercatch(row['prcp'], row['CE_sc_reanalysis']), axis=1)

                df['CE_eccc_reanalysis'] = df.apply(lambda row: calculate_CE(row['reanalysis_wind'], row['Tair(C)']), axis=1)
                df['corrected_prcp_eccc_reanalysis'] = df.apply(lambda row: apply_undercatch(row['Unadj_P(mm)'], row['CE_eccc_reanalysis']), axis=1)


                # Save the updated DataFrame back to the CSV file
                df.to_csv(file_path, index=False)
                print(f"Undercatch calculated for {file_path}")

# Process the CSV files in the directory
calculate_undercatch_for_gsod_stations_with_reanalysis(merged_station_path)

NameError: name 'merged_station_path' is not defined

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def generate_accumulated_precipitation_plots_per_water_year(input_csv_path,station,output_dir):
    df = pd.read_csv(input_csv_path)
    
    # Convert date column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Add a water year column
    df['water_year'] = df['date'].apply(lambda x: x.year if x.month < 10 else x.year + 1)
    
    # Get unique water years
    water_years = df['water_year'].unique()
    
    # Calculate the number of rows and columns for subplots
    num_plots = len(water_years)
    num_cols = math.ceil(math.sqrt(num_plots))
    num_rows = math.ceil(num_plots / num_cols)

    # Create a figure with subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 6 * num_rows), sharex=False)

    # Flatten axes array for easy iteration
    if num_plots == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    # Calculate accumulated precipitation for each water year and plot
    for ax, water_year in zip(axes, water_years):
        group = df[df['water_year'] == water_year].sort_values(by='date')
        group['Accum_Unadj_P(mm)'] = group['Unadj_P(mm)'].cumsum()
        group['Accum_prcp'] = group['prcp'].cumsum()
        group['Accum_UTF_Adj_P(mm)'] = group['UTF_Adj_P(mm)'].cumsum()
        group['Accum_corrected_prcp_sc_earth'] = group['corrected_prcp_sc_earth'].cumsum()
        group['Accum_corrected_prcp_sc_reanalysis'] = group['corrected_prcp_sc_reanalysis'].cumsum()
        group['Accum_corrected_prcp_eccc_reanalysis'] = group['corrected_prcp_eccc_reanalysis'].cumsum()

        ax.plot(group['date'], group['Accum_Unadj_P(mm)'], label='Raw Gauge (ECCC)', linestyle='-', color='red')
        ax.plot(group['date'], group['Accum_prcp'], label='Raw Gauge (SC-Earth)', linestyle='-', color='blue')
        ax.plot(group['date'], group['Accum_UTF_Adj_P(mm)'], label='Undercatch Corrected (ECCC)', linestyle=':', color='red')
        ax.plot(group['date'], group['Accum_corrected_prcp_sc_earth'], label='Undercatch Corrected (SC-Earth)', linestyle=':', color='blue')
        ax.plot(group['date'], group['Accum_corrected_prcp_sc_reanalysis'], label='Undercatch Corrected (SC-Earth+ERA5 Wind)', linestyle=':', color='purple')
        ax.plot(group['date'], group['Accum_corrected_prcp_eccc_reanalysis'], label='Undercatch Corrected (ECCC+ERA5 Wind)', linestyle=':', color='green')
        ax.set_xlabel('Month')
        ax.set_ylabel('Accumulated Precipitation (mm)')
        ax.set_title(f'Accumulated Precipitation (Water Year {water_year})')
        ax.grid(True)

        # Format x-axis to display only the month
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

        # Maintain the right x-axis bound
        ax.set_xlim([group['date'].min(), group['date'].max()])

    # Hide any unused subplots
    for i in range(num_plots, num_rows * num_cols):
        fig.delaxes(axes[i])

    # Add a single legend at the bottom of the figure
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=3)

    # Adjust layout to prevent overlap
    plt.tight_layout(rect=[0, 0.1, 1, 1])  # Adjust rect to make space for the legend

    # Save the combined plot to a PNG file
    output_plot_path = os.path.join(output_dir, f'{station}_accumulated_precipitation.png')
    print(f"Saving plot to {output_plot_path}")
    plt.savefig(output_plot_path)
    plt.close()

# Run the function to generate the plots

for station in station_name_list:
    station = str(station)
    comparative_file = Path(merged_station_path,f'{station}.csv')

    generate_accumulated_precipitation_plots_per_water_year(comparative_file,station,plot_path)



Saving plot to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_accumulated_precipitation.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_accumulated_precipitation.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_accumulated_precipitation.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_accumulated_precipitation.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/REVELSTOKEAIRPORTAUTO_accumulated_precipitation.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/ROCKYMTNHOUSE(AUT)_accumulated_precipitation.png


In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_ce_boxplot(file_path,station,output_dir,legend_title='Method', method_labels=None):
    """
    Plots a boxplot of CE for each method aggregated by month.

    Parameters:
    - file_path: str, path to the CSV file
    - legend_title: str, title for the legend
    - method_labels: dict, dictionary to map method column names to desired labels
    """
    # Load the CSV file
    data = pd.read_csv(file_path)

    # Convert the date column to datetime format
    data['date'] = pd.to_datetime(data['date'])

    # Extract the month from the date column
    data['month'] = data['date'].dt.month

    # Melt the dataset to have one column for method and another for CE values
    melted_data = pd.melt(data, id_vars=['month'], value_vars=['CE', 'CE_sc_earth','CE_sc_reanalysis','CE_eccc_reanalysis'], var_name='method', value_name='CE_calc')
    #Drop all values where CE is 1
    melted_data = melted_data[melted_data['CE_calc'] != 1]

    # Apply method labels if provided
    if method_labels:
        melted_data['method'] = melted_data['method'].map(method_labels)

    # Create the boxplot
    plt.figure(figsize=(14, 8))
    sns.boxplot(x='month', y='CE_calc', hue='method', data=melted_data)
    plt.title(f'Boxplot of CE for {station}')
    plt.xlabel('Month')
    plt.ylabel('CE')
    plt.legend(title=legend_title)
    plt.grid(True)
    # Save the plot to a PNG file
    output_plot_path = f'{output_dir}/{station}_CE.png'
    print(f'Saving plot to {output_plot_path}')
    plt.savefig(output_plot_path)
    plt.close()

# Usage example
for station in station_name_list:
    station = str(station)
    comparative_file = Path(merged_station_path,f'{station}.csv')
    
    method_labels = {'CE': 'ECCC Analysis', 'CE_sc_earth': 'SC-Earth','CE_eccc_reanalysis': 'ECCC-ERA5','CE_sc_reanalysis': 'SC-Earth-ERA5'}
    plot_ce_boxplot(comparative_file,station,plot_path,legend_title='CE Methods', method_labels=method_labels)

Saving plot to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_CE.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_CE.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_CE.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_CE.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/REVELSTOKEAIRPORTAUTO_CE.png
Saving plot to /Users/dcasson/Data/pems/undercatch/plots/ROCKYMTNHOUSE(AUT)_CE.png


In [34]:
import pandas as pd
from scipy import stats

def generate_comparative_plots(file_path, col1, col2, output_dir, station_name, x_label=None, y_label=None):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Plot correlation with regression line
    plt.figure(figsize=(10, 6))
    sns.regplot(x=col1, y=col2, data=df, line_kws={"color": "red", "alpha": 0.7, "lw": 2})
    
    # Calculate correlation coefficient and regression equation
    slope, intercept, r_value, p_value, std_err = stats.linregress(df[col1], df[col2])
    regression_formula = f'Y = {intercept:.2f} + {slope:.2f}X'
    correlation_coefficient = f'R^2 = {r_value**2:.2f}'
    
        # Add a 1:1 line for reference
    max_val = max(df[col1].max(), df[col2].max())
    min_val = min(df[col1].min(), df[col2].min())
    plt.plot([min_val, max_val], [min_val, max_val], color='blue', linestyle='--', linewidth=1, label='1:1 Line')
    
    # Annotate plot with regression equation and R^2
    plt.text(0.05, 0.95, regression_formula, transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
    plt.text(0.05, 0.90, correlation_coefficient, transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
    
    # Set labels
    plt.title(f'Correlation between {col1} and {col2}')
    plt.xlabel(x_label if x_label else col1)
    plt.ylabel(y_label if y_label else col2)
    plt.grid(True)
    
    # Define the output file path
    output_file = os.path.join(output_dir, f'{station_name}_{col2}.png')
    
    # Save the plot to the specified directory
    plt.savefig(output_file)
    plt.close()
    print(f"Plot saved to {output_file}")

for station in station_name_list:
    station = str(station)
    comparative_file = Path(merged_station_path,f'{station}.csv')
    generate_comparative_plots(comparative_file, 'Wind(m/s)', 'wind',plot_path, station, x_label='ECCC Wind', y_label='EC-Earth Wind')
    generate_comparative_plots(comparative_file, 'Tair(C)', 'tmean', plot_path, station, x_label='ECCC Temperature (°C)', y_label='EC-Earth Temperature (°C)')
    generate_comparative_plots(comparative_file, 'Wind(m/s)', 'reanalysis_wind',plot_path, station, x_label='ECCC Wind', y_label='reanalysis Wind')
    #generate_comparative_plots(comparative_file, 'Tair(C)', 'tmean', plot_path, station, x_label='ECCC Temperature (°C)', y_label='EC-Earth Temperature (°C)')

    

Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_tmean.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_reanalysis_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_tmean.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_reanalysis_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_tmean.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_reanalysis_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_wind.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_tmean.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_reanalysis_wind.png
Plot saved to /User

In [36]:
def plot_histogram(df, columns,labels,plot_path,station_name, bins=30):
    """
    Plots overlapping histograms for the specified columns in the dataframe.
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    columns (list): List of columns to plot.
    bins (int): Number of bins for the histogram.
    """
    plt.figure(figsize=(12, 8))
    
    for column,label in zip(columns, labels):
        plt.hist(df[column], bins=bins, alpha=0.5, label=label)
    
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title(f'Station {station_name} Wind Distribution')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    #plt.show()
    # Define the output file path
    output_file = os.path.join(plot_path, f'{station_name}_wind_distribution.png')
    
    # Save the plot to the specified directory
    plt.savefig(output_file)
    plt.close()
    print(f"Plot saved to {output_file}")

# Plot overlapping histograms for 'Wind(m/s)', 'wind', and 'reanalysis_wind'
columns_to_plot = ['Wind(m/s)', 'wind', 'reanalysis_wind']
labels = ['ECCC Wind', 'EC-Earth Wind', 'Reanalysis Wind']
for station in station_name_list:
    station = str(station)
    comparative_file = Path(merged_station_path,f'{station}.csv')
    df = pd.read_csv(comparative_file)
    plot_histogram(df, columns_to_plot,labels, plot_path, station)

Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/REVELSTOKEAIRPORTAUTO_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/ROCKYMTNHOUSE(AUT)_wind_distribution.png


In [46]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def plot_histogram(df, columns, labels, plot_path, station_name, bins):
    """
    Plots overlapping histograms for the specified columns in the dataframe using the provided bins.
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    columns (list): List of columns to plot.
    bins (array): Array of bin edges for the histogram.
    """
    plt.figure(figsize=(12, 8))
    
    for column, label in zip(columns, labels):
        plt.hist(df[column], bins=bins, alpha=0.5, label=label)
    
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title(f'Station {station_name} Wind Distribution')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    # Define the output file path
    output_file = os.path.join(plot_path, f'{station_name}_wind_distribution.png')
    
    # Save the plot to the specified directory
    plt.savefig(output_file)
    plt.close()
    print(f"Plot saved to {output_file}")

def determine_common_bins(df, columns, bins=30):
    """
    Determines common bin edges for all columns in the dataframe.
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    columns (list): List of columns to consider for bin edges.
    bins (int): Number of bins for the histogram.
    
    Returns:
    array: Array of bin edges.
    """
    min_value = df[columns].min().min()
    max_value = df[columns].max().max()
    bin_edges = np.linspace(min_value, max_value, bins + 1)
    return bin_edges

station_name_list = read_station_id_list(evaluation_stations_csv, 'Name')
station_name_list = [s.replace(' ', '') for s in station_name_list]
columns_to_plot = ['Wind(m/s)', 'wind', 'reanalysis_wind']
labels = ['ECCC Wind', 'EC-Earth Wind', 'Reanalysis Wind']

# Plot histograms for each station
for station in station_name_list:
    station = str(station)
    comparative_file = Path(merged_station_path, f'{station}.csv')
    df = pd.read_csv(comparative_file)
    common_bins = determine_common_bins(df, columns_to_plot)
    plot_histogram(df, columns_to_plot, labels, plot_path, station, common_bins)

Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BANFFCS_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/BOWVALLEY_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/JASPERWARDEN_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/PINCHERCREEKCLIMATE_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/REVELSTOKEAIRPORTAUTO_wind_distribution.png
Plot saved to /Users/dcasson/Data/pems/undercatch/plots/ROCKYMTNHOUSE(AUT)_wind_distribution.png
